In [3]:
import sys
import json
import random
sys.version

'3.7.4 (default, Oct 15 2019, 22:29:14) \n[GCC 7.4.0]'

In [6]:
from py2neo import Graph
from neo4j import GraphDatabase
NEO4J_URI="bolt://localhost:7687"
SC_JSON = "semantic-scholar-raw/s2-corpus-slim.json"
SC_JSON_GITHUB = "https://raw.githubusercontent.com/diegoquintanav/od-lab-1/master/notebooks/semantic-scholar-raw/s2-corpus-slim.json"

graph = Graph(NEO4J_URI)
gdb = GraphDatabase.driver(uri=NEO4J_URI, auth=None)

In [7]:
from faker import Faker
from faker.providers import lorem
fake = Faker()
fake.seed_instance(42)
fake.add_provider(lorem)

http://s2-public-api-prod.us-west-2.elasticbeanstalk.com/corpus/

Download a part of the corpus with the `aws` command line interface, like

```aws s3 cp --no-sign-request s3://ai2-s2-research-public/open-corpus/2018-05-03/s2-corpus-00.gz .```

or

```wget https://s3-us-west-2.amazonaws.com/ai2-s2-research-public/open-corpus/2018-05-03/s2-corpus-00.gz```

And then `gunzip` it.

In [31]:
!ls semantic-scholar-raw/

license.txt   s2-corpus-00	    s2-corpus-clean.tmp
manifest.txt  s2-corpus-clean.json  s2-corpus-slim.json


In [32]:
!wc -l semantic-scholar-raw/s2-corpus-00

1000000 semantic-scholar-raw/s2-corpus-00


Filter out those lines that have empty entities, empty inCitations, and empty outCitations

In [23]:
!grep -v -e '"entities":\[\]' -e '"inCitations":\[\]' -e '"outCitations":\[\]' semantic-scholar-raw/s2-corpus-00 > semantic-scholar-raw/s2-corpus-clean.tmp
!wc -l semantic-scholar-raw/s2-corpus-clean.tmp

212901 semantic-scholar-raw/s2-corpus-clean.tmp


Filter out those rows with empty attributes

In [28]:
!grep -v -e '"journalName":""' -e '"title":""' -e '"journalVolume":""' -e '"id":""' -e '"ids":""' -e '"doi":""' -e '"doiUrl":""' semantic-scholar-raw/s2-corpus-clean.tmp > semantic-scholar-raw/s2-corpus-clean.json
!wc -l semantic-scholar-raw/s2-corpus-clean.json

86609 semantic-scholar-raw/s2-corpus-clean.json


Obtain a smaller sample from that data

In [29]:
!head -100 semantic-scholar-raw/s2-corpus-clean.json > semantic-scholar-raw/s2-corpus-slim.json

In [27]:
!head -1 semantic-scholar-raw/s2-corpus-slim.json | jq

{
  "entities": [
    "Eating",
    "Ethanol",
    "Isopropyl Alcohol 0.7 ML/ML Medicated Pad"
  ],
  "journalVolume": "31 4",
  "journalPages": "385-8",
  "pmid": "8879286v1",
  "year": 1996,
  "outCitations": [
    "40bb6636803d10626bfafa3dbf940fdac903d11c",
    "18ffb4ae9e0d8f0c43342c4c6b993f3ce34bd579",
    "827b793fbc923d3e068b2008dc02bb63e7069fb9",
    "b96cb6dd0bba1e9928eb6315e18339f667c6799f",
    "48431576480666445bdcb112533aa885d4ac1ad1",
    "1380d901dbce2dca8eba90501179485d4fe1e5d4",
    "4203ad4dc8530a92473e5576650eca84b51c9eee",
    "96b85a405b002e11199f967280ec9a9736543c86",
    "8d72d01b1b670c7bb65408e91d1705f0c95b1174",
    "7842615744a118a2d4a239787e9450c26c98c9f9",
    "6ed7439f9537fce3f82144843622b9c670a95d7e",
    "55787878e91f6efdf14e16438e733d80581f3bec",
    "e28468ccfa0ddce1d060d39d25305f6abab2eced",
    "5f6ba13f126408d25849c518bddf4812281ae970"
  ],
  "s2Url": "https://semanticscholar.org/paper/30aaf92bde9bf6cdc3dff237691fd5db55d4cb4a",
  "s2PdfUrl": "http://

The final modified JSON is accesible now from github

In [72]:
myjson = "https://raw.githubusercontent.com/diegoquintanav/od-lab-1/master/notebooks/semantic-scholar-raw/s2-corpus-slim.json"

## A.2 Loading

In [191]:
graph.run("MATCH (n) DETACH DELETE n").stats()

constraints_added: 0
constraints_removed: 0
contained_updates: True
indexes_added: 0
indexes_removed: 0
labels_added: 0
labels_removed: 0
nodes_created: 0
nodes_deleted: 3150
properties_set: 0
relationships_created: 0
relationships_deleted: 8766

We will be using apoc (https://github.com/neo4j-contrib/neo4j-apoc-procedures)

In [166]:
graph.run("RETURN apoc.version();").data()

[{'apoc.version()': '3.5.0.9'}]

In [167]:
graph.run("call dbms.components() yield name, versions, edition unwind versions as version return name, version, edition;").data()

[{'name': 'Neo4j Kernel', 'version': '3.5.15', 'edition': 'enterprise'}]

In [168]:
query = """
WITH $myjson AS url 
CALL apoc.load.json(url) 
YIELD value 
RETURN value 
LIMIT 1"""

graph.run(query, myjson = myjson).data()

[{'value': {'doiUrl': 'https://doi.org/10.1007/BF02270828',
   'venue': 'Evolutionary Ecology',
   'journalName': 'Evolutionary Ecology',
   'sources': [],
   'year': 2005,
   'paperAbstract': 'I compare the relationship between bill size (depth) and body size among different taxa of seed-eating finches to test the hypothesis (Schluter, 1988a) that in habitats where seed-eating finches are vulnerable to predators, finches have larger bodies relative to their bill size. In support of this hypothesis, ground-foraging finches on continents (Emberizidae, Passeridae, Fringillidae), where predators are more common, have larger bodies relative to their bill sizes than do ground-foraging finches on islands (Emberizidae, Fringillidae). Ground-feeding finches on continents may also be more vulnerable to predators than tree- and shrub-foraging finches. As predicted, in North America, ground-feeding finches (Emberizinae) have larger bodies relative to their bill size than do tree- and shrub-foragi

Here, `"myjson"` is an alias for the path to the file, located in github

### Create Article instances

In [197]:
query_apoc = """WITH $myjson AS url 
CALL apoc.load.json(url) 
YIELD value as row
MERGE (art:Article {
        title: row.title,
        id: row.id, doi: 
        row.doi, 
        doi_url:row.doiUrl,
        year: row.year})
MERGE (art)-[:PUBLISHED_IN {pages: row.journalPages}]->(vol:Volume {volume: row.journalVolume, year: row.year})-[:OF]->(jour:Journal {
            name: row.journalName})
FOREACH (author in row.authors |
    MERGE (auth:Author {name: author.name, id: author.ids})-[:AUTHORS]->(art)
)
FOREACH (k in row.entities |
    MERGE (art)-[:CONTAINS]->(kw: Keyword {name: k}))"""

graph.run(query_apoc, myjson = myjson).stats()

constraints_added: 0
constraints_removed: 0
contained_updates: True
indexes_added: 0
indexes_removed: 0
labels_added: 1998
labels_removed: 0
nodes_created: 1998
nodes_deleted: 0
properties_set: 3089
relationships_created: 1898
relationships_deleted: 0

Some data is left to be simulated

### Faking citations

In [198]:
q_article_ids = "MATCH (n:Article) RETURN n.id"
article_ids = [v["n.id"] for v in graph.run(q_article_ids).data()]
n_articles = len(article_ids)
article_ids[:5]

['ed6f7f0d65f4c8bde43f13667c406ff3403f9814',
 'bfea5a8c0ed0bc15ff8365e39c8035f7eb7f50d3',
 'b8be59215c7f983bd1e3fbfc65e441c3ac2c618d',
 '36efcd42e8fbe18e0e1baf6c2696e2971aee78b2',
 '864ca4044d2ebf88ae4bf45df730f571039488b3']

In [199]:
# https://neo4j.com/docs/driver-manual/1.7/sessions-transactions/#driver-transactions-transaction-functions
q_add_citation_rel_id = """MATCH (a:Article),(b:Article)
WHERE a.id = $id_a AND b.id = $id_b
MERGE p=(a)-[r:CITED_BY]->(b)
RETURN p"""

def add_citation_rel(driver, id_a, id_b):
    with driver.session() as session:
        # Caller for transactional unit of work
        return session.write_transaction(create_citation_rel, id_a, id_b)

# Simple implementation of the unit of work
def create_citation_rel(tx, id_a, id_b):
    return tx.run(q_add_citation_rel_id, id_a=id_a, id_b = id_b)

Add 500 relationships of type `cited_by`

In [200]:
random.seed(42)
# pick a sample of 500 papers, and make them be cited by other three papers at random
created_citations = []

for article in article_ids:
    for citation in random.sample(article_ids, random.randint(5, 50)):
        if article != citation: # they can't cite themselves
            _out = add_citation_rel(gdb, id_a=article, id_b=citation)
            created_citations.append(_out.data())
        else:
            print("Skipping: ", article, citation)

created_citations[:5]

Skipping:  ed6f7f0d65f4c8bde43f13667c406ff3403f9814 ed6f7f0d65f4c8bde43f13667c406ff3403f9814
Skipping:  ed26a77ce2ffaa0ea2edd17afdd52f5642574b6f ed26a77ce2ffaa0ea2edd17afdd52f5642574b6f
Skipping:  f0888ca406fc5ea91e8d0212052b781b8a930cf3 f0888ca406fc5ea91e8d0212052b781b8a930cf3
Skipping:  0d890bec1cf9adedbe8683e0f7b7ec208cb35ed3 0d890bec1cf9adedbe8683e0f7b7ec208cb35ed3
Skipping:  acaa12d900e5a682f399162091845f8ad47a03f4 acaa12d900e5a682f399162091845f8ad47a03f4
Skipping:  b67f04742496fb29ebc02de237b6fa039efcae4b b67f04742496fb29ebc02de237b6fa039efcae4b
Skipping:  b4253ff9f61743fab39342cb4e47b6f9b88e004c b4253ff9f61743fab39342cb4e47b6f9b88e004c
Skipping:  a17699a73ac3c6dc8bcec10ecc420857d88eb632 a17699a73ac3c6dc8bcec10ecc420857d88eb632
Skipping:  a09c196f66e8b5f44a7b734862a2a00e66d079a5 a09c196f66e8b5f44a7b734862a2a00e66d079a5
Skipping:  9df19587849ca5cd98e43b6072aaa122efb5a29f 9df19587849ca5cd98e43b6072aaa122efb5a29f
Skipping:  11781ea072a749847253639a9903e8b796d51e54 11781ea072a7498472

[[{'p': <Path start=<Node id=26133 labels={'Article'} properties={'doi_url': 'https://doi.org/10.1007/BF02270828', 'id': 'ed6f7f0d65f4c8bde43f13667c406ff3403f9814', 'title': 'Predation, seed size partitioning and the evolution of body size in seed-eating finches', 'year': 2005, 'doi': '10.1007/BF02270828'}> end=<Node id=26336 labels={'Article'} properties={'doi_url': 'https://doi.org/10.1109/TSMCC.2005.848191', 'id': '02fc5fada3a31fc2b128b4784fd2e8c8a75b8b08', 'title': 'An adaptive multimodal biometric management algorithm', 'year': 2005, 'doi': '10.1109/TSMCC.2005.848191'}> size=1>}],
 [{'p': <Path start=<Node id=26133 labels={'Article'} properties={'doi_url': 'https://doi.org/10.1007/BF02270828', 'id': 'ed6f7f0d65f4c8bde43f13667c406ff3403f9814', 'title': 'Predation, seed size partitioning and the evolution of body size in seed-eating finches', 'year': 2005, 'doi': '10.1007/BF02270828'}> end=<Node id=26204 labels={'Article'} properties={'doi_url': 'https://doi.org/10.1007/s00439-003-1

### Faking reviewers 

We will create a `REVIEWS` relationship between authors and articles

In [201]:
# delete relationships
graph.run("MATCH p=(:Author)-[r:REVIEWS]->(:Article) DELETE r RETURN COUNT(r)").stats()

constraints_added: 0
constraints_removed: 0
contained_updates: False
indexes_added: 0
indexes_removed: 0
labels_added: 0
labels_removed: 0
nodes_created: 0
nodes_deleted: 0
properties_set: 0
relationships_created: 0
relationships_deleted: 0

In [202]:
q_author_ids = "MATCH (n:Author) RETURN n.id"
author_ids = [v["n.id"] for v in graph.run(q_author_ids).data()]    
author_ids[:5]

[['21471045'], ['21685210'], ['21131969'], ['6569619'], ['5194599']]

#### Assign reviewers randomly to 1000 articles


In [203]:
# assign reviewers randomly to 1000 articles
random.seed(42)

q_add_reviewers = """MATCH (a:Article),(b:Author)
WHERE a.id = $article_id AND b.id = $author_id
AND NOT (b)-[:AUTHORS]->(a)
MERGE p=(b)-[:REVIEWS]->(a)
RETURN p"""

created_review_rel = []

with gdb.session() as session:
    for article_id in article_ids:
        for author_id in random.sample(author_ids, random.choice([2, 6])):
            _out = graph.run(q_add_reviewers, article_id=article_id, author_id=author_id)
            created_review_rel.append(_out.data())
            
created_review_rel[:5]

[[{'p': (Christoph Vauth)-[:REVIEWS {}]->(_26133)}],
 [{'p': (Serhan Karvar)-[:REVIEWS {}]->(_26133)}],
 [{'p': (Heinz Künzle)-[:REVIEWS {}]->(_26186)}],
 [{'p': (Angel Gomez-Sacristan)-[:REVIEWS {}]->(_26186)}],
 [{'p': (Yi Liu)-[:REVIEWS {}]->(_26186)}]]

### Fake conferences and editions

In [204]:
# delete conference and editions
graph.run("MATCH (n:Conference) DETACH DELETE n RETURN COUNT(n) as deleted_conferences").data()

[{'deleted_conferences': 0}]

In [205]:
graph.run("MATCH (n:Edition) DETACH DELETE n RETURN COUNT(n) as deleted_editions").data()


[{'deleted_editions': 0}]

We will fake 600 editions, distributed between 100 conferences

In [206]:
random.seed(42)
N_CONFERENCES = 100

fake_conferences = [{
    "id": idx, 
    "name": fake.name()}
    for idx, _ in enumerate(range(100), start=1)]

fake_editions = [
    {
        "id": idx, 
        "venue": fake.name(),
        "city": fake.city(),
        "year": fake.year()}
    for idx, _ in enumerate(range(600), start=1)]

# assign 4 to 20 editions to each conference
for conference in fake_conferences:
    conference["editions"] = []
    for edition in random.sample(fake_editions, random.randint(4, 20)):
        conference["editions"].append(edition)

fake_conferences[:2]

[{'id': 1,
  'name': 'Donna Salazar',
  'editions': [{'id': 26,
    'venue': 'Samantha Edwards',
    'city': 'North Sandra',
    'year': '1986'},
   {'id': 282,
    'venue': 'Douglas Berry',
    'city': 'New Williamfort',
    'year': '1984'},
   {'id': 251,
    'venue': 'Richard Brown',
    'city': 'Michelleside',
    'year': '1987'},
   {'id': 229,
    'venue': 'Nicholas Griffith',
    'city': 'Cantrellburgh',
    'year': '2004'},
   {'id': 143,
    'venue': 'Elizabeth Howell',
    'city': 'Smithbury',
    'year': '2005'},
   {'id': 105,
    'venue': 'Andrea Cline',
    'city': 'South Thomas',
    'year': '1984'},
   {'id': 559,
    'venue': 'Kristina Chavez',
    'city': 'Ryanbury',
    'year': '2003'}]},
 {'id': 2,
  'name': 'Richard Conley',
  'editions': [{'id': 433,
    'venue': 'Amber Rivera',
    'city': 'Jenniferfurt',
    'year': '2013'},
   {'id': 33,
    'venue': 'Bradley Moon',
    'city': 'North Amandaview',
    'year': '1984'},
   {'id': 31,
    'venue': 'Denise Reyes',


In [207]:
q_create_conference = "MERGE (c:Conference {id: $cid, name: $name}) RETURN c"
q_create_edition = "MERGE (e:Edition {id: $eid, conference_id: $cid, venue: $venue, city: $city, year: $year}) RETURN e"


q_merge_conference_and_edition = """MATCH (c:Conference), (e:Edition)
WHERE c.id = $cid AND e.id = $eid
MERGE p=(c)-[:IN]->(e)
RETURN p"""

created_conferences = []

with gdb.session() as session:
    for conference in fake_conferences:
        # create conference
        session.run(q_create_conference, cid = conference["id"], name = conference["name"])
        for edition in conference["editions"]:
            # create edition
            session.run(q_create_edition,
                        eid = edition["id"],
                        cid = conference["id"], 
                        venue = edition["venue"], 
                        city = edition["city"], 
                        year = edition["year"])
            # assign it to conference
            _out = session.run(q_merge_conference_and_edition, eid = edition["id"], cid = conference["id"])
            created_conferences.append(_out.data())

In [208]:
created_conferences[:5]

[[{'p': <Path start=<Node id=8225 labels={'Conference'} properties={'name': 'Donna Salazar', 'id': 1}> end=<Node id=8228 labels={'Edition'} properties={'venue': 'Samantha Edwards', 'conference_id': 1, 'id': 26, 'city': 'North Sandra', 'year': '1986'}> size=1>}],
 [{'p': <Path start=<Node id=8225 labels={'Conference'} properties={'name': 'Donna Salazar', 'id': 1}> end=<Node id=7635 labels={'Edition'} properties={'venue': 'Douglas Berry', 'conference_id': 1, 'id': 282, 'city': 'New Williamfort', 'year': '1984'}> size=1>}],
 [{'p': <Path start=<Node id=8225 labels={'Conference'} properties={'name': 'Donna Salazar', 'id': 1}> end=<Node id=7637 labels={'Edition'} properties={'venue': 'Richard Brown', 'conference_id': 1, 'id': 251, 'city': 'Michelleside', 'year': '1987'}> size=1>}],
 [{'p': <Path start=<Node id=8225 labels={'Conference'} properties={'name': 'Donna Salazar', 'id': 1}> end=<Node id=7638 labels={'Edition'} properties={'venue': 'Nicholas Griffith', 'conference_id': 1, 'id': 229,

### Assigning articles to editions

And we will assign articles randomly to 2 to 10 editions

In [209]:
fake_editions[:1]

[{'id': 1,
  'venue': 'Angela Stephens',
  'city': 'Lake Danielfurt',
  'year': '2013'}]

In [210]:
# assign each paper to 2-10 editions
random.seed(42)

q_link_to_conf = """MATCH (a:Article), (b:Edition)
WHERE a.id = $aid AND b.id = $eid
MERGE p=(a)-[r:PUBLISHED_IN]->(b)
RETURN p"""

created_art_ed_rel = []

with gdb.session() as session:
    for aid in article_ids:
        for edition in random.sample(fake_editions, random.randint(2, 10)):
            _out = session.run(q_link_to_conf, aid = aid, eid = edition["id"])
            created_art_ed_rel.append(_out.data())
            
created_art_ed_rel[:1]

[[{'p': <Path start=<Node id=26133 labels={'Article'} properties={'doi_url': 'https://doi.org/10.1007/BF02270828', 'id': 'ed6f7f0d65f4c8bde43f13667c406ff3403f9814', 'title': 'Predation, seed size partitioning and the evolution of body size in seed-eating finches', 'year': 2005, 'doi': '10.1007/BF02270828'}> end=<Node id=8228 labels={'Edition'} properties={'venue': 'Samantha Edwards', 'conference_id': 1, 'id': 26, 'city': 'North Sandra', 'year': '1986'}> size=1>},
  {'p': <Path start=<Node id=26133 labels={'Article'} properties={'doi_url': 'https://doi.org/10.1007/BF02270828', 'id': 'ed6f7f0d65f4c8bde43f13667c406ff3403f9814', 'title': 'Predation, seed size partitioning and the evolution of body size in seed-eating finches', 'year': 2005, 'doi': '10.1007/BF02270828'}> end=<Node id=8294 labels={'Edition'} properties={'venue': 'Samantha Edwards', 'conference_id': 33, 'id': 26, 'city': 'North Sandra', 'year': '1986'}> size=1>},
  {'p': <Path start=<Node id=26133 labels={'Article'} propertie

## A.3 Evolving the graph 



### Store the review and the approval sent by each reviewer

Since the queries in part B don't need this information, we can store the `review_decision` and the `review_text` as _edge_ attributes of the relation `REVIEWED_BY`.


Alternatively, and in the case these reviews were a requirement in part B, they can be implemented by creating a `Review` node, and linking it to articles, in a four step process.

1. Create a `Review` article for each `Article` that has reviewers (i.e. is connected to authors by `reviewed_by` edges). Each `Review` node will have an attribute named `review_contents´.
2. Attach review to the article
3. Attach reviewers to the review instance
4. Optional, delete the `reviewed_by` edges from the step 1.

In [211]:
with gdb.session() as session:
    out = session.run("MATCH p=(:Author)-[r:REVIEWS]->(:Article) return ID(r)")
    rel_ids = [r[0] for r in out.values()]
    
rel_ids[:5]

[1104267, 1104266, 1104268, 1104269, 1104270]

In [212]:
q_merge_review_attributes = """MATCH p=(:Author)-[r:REVIEWS]->(:Article)
WHERE ID(r) = $rel_id
SET r.review_accepted = $review_accepted, r.review_text = $review_text
RETURN r
"""

created_review_rel_attr = []

random.seed(42)

with gdb.session() as session:
    for rel_id in rel_ids:
        _out = session.run(q_merge_review_attributes, 
                    rel_id=rel_id,
                    review_accepted=fake.pybool(), 
                    review_text=fake.texts(nb_texts=1, max_nb_chars=500)[0])
        created_review_rel_attr.append(_out.data())

In [213]:
created_review_rel_attr[0]

[{'r': <Relationship id=1104267 nodes=(<Node id=5541 labels=set() properties={}>, <Node id=26133 labels=set() properties={}>) type='REVIEWS' properties={'review_text': 'About budget international knowledge product. Expert issue wonder. Help party marriage attorney its.\nNetwork successful quite. Body behavior figure reveal break. Mention hospital subject couple.\nAll success we lose affect art. Style fall prevent affect system.\nFeeling hit item order away every recently thousand. Your break visit open. These since speech.\nNetwork process over almost. Could be as allow before work hit.', 'review_accepted': True}>}]

### Store a reviewing policy for each Journal or Conference

This can also be implemented as an `review_policy_min_count` to the `Journal` and `Proceedings` labels. We will set all of them to 3.

In [214]:
with gdb.session() as session:
    journal_ids = session.run("MERGE (n:Journal) SET n.review_policy_min_count = 3 return n").values()
    proceeding_ids = session.run("MERGE (n:Conference) SET n.review_policy_min_count = 3 return n").values()

In [215]:
journal_ids[:5]

[[<Node id=27683 labels={'Journal'} properties={'review_policy_min_count': 3, 'name': 'Evolutionary Ecology'}>],
 [<Node id=27787 labels={'Journal'} properties={'review_policy_min_count': 3, 'name': 'Emergency medicine Australasia : EMA'}>],
 [<Node id=27816 labels={'Journal'} properties={'review_policy_min_count': 3, 'name': 'The American journal of surgical pathology'}>],
 [<Node id=27840 labels={'Journal'} properties={'review_policy_min_count': 3, 'name': 'Human Genetics'}>],
 [<Node id=27924 labels={'Journal'} properties={'review_policy_min_count': 3, 'name': 'International journal of cardiology'}>]]

In [216]:
proceeding_ids[:2]

[[<Node id=7642 labels={'Conference'} properties={'review_policy_min_count': 3, 'name': 'Richard Conley', 'id': 2}>],
 [<Node id=7652 labels={'Conference'} properties={'review_policy_min_count': 3, 'name': 'Timothy Taylor', 'id': 3}>]]

### Display the affiliation of the author to an organization or a company 

This can be solved by creating an `affiliation_institution_type` attribute, and also an `affiliation_institution_name`. Note that we could have created these as nodes, but the queries from part B do not refer to this information.


In [217]:
q_set_affiliations = """MATCH (n:Author)
WHERE n.id = $author_id
SET n.affiliation_institution_type = $affiliation_institution_type, n.affiliation_institution_name = $affiliation_institution_name
RETURN n
"""

institution_types = ['University', 'Company', 'NGO']
created_affiliations = []

random.seed(42)

with gdb.session() as session:
    for author_id in author_ids:
        _out = session.run(q_set_affiliations, 
                    author_id=author_id,
                    affiliation_institution_name=fake.company(), 
                    affiliation_institution_type=random.choice(institution_types))        
        created_affiliations.append(_out.values())